# 성능개선법

### 1. Model selection(볼드는 시도해본 것, 기울임은 시도해볼만한 것)
> *RNN&LSTM&GRU는 basemodel에서 이름만 바꿔주면 되더라... 맞는 사용법인지는 모름...*

* 머신러닝: 알아는 보았지만 시계열 예측에 쓰이는지는 모르겠다. 분류도 엉망진창
  * Multi-Layer Perceptron (MLP)
  * Bayesian Neural Network (BNN)
  * Radial Basis Functions (RBF)
  * Generalized Regression Neural Networks (GRNN)
  * kernel regression K-Nearest Neighbor regression (KNN)
  * CART regression trees (CART)
  * Support Vector Regression (SVR)
  * Gaussian Processes (GP)

* 딥러닝: 대체로 RNN 기반만 시계열 예측에 쓰이는 것 같다.
  * **Recurrent Neural Network (RNN)** : base model과 비교하여 비슷하거나 더 낮은 성능을 보임.
  * **Long Short-Term Memory (LSTM)** : base model.
  * **Gated Recurrent Unit (GRU)** : base model과 비교하여 비슷하거나 더 낮은 성능을 보임.
  * *grid LSTM*
  * *Transfomer*
    * Transformer 계열의 LittleBird 모델
    * Time Series Forecasting_Transfomer -> https://doheon.github.io/%EC%BD%94%EB%93%9C%EA%B5%AC%ED%98%84/time-series/ci-4.transformer-post/
    * Transfomer를 통한 시계열 데이터 예측 -> https://aboutnlp.tistory.com/55

    * 앙상블(Ensemble) 기법: 쓸만한 모델들이 추려지면...
      * Boost 기법
        > XG부스트, 라이트GBM(LightGBM), 캣부스트(CatBoost)

    ? 잘 모르겠음: Random Forest, Decision Tree, fbprophet- predict...


### 2. Hyperparameter Tuning
* Hyperparameter
  * Epochs
  * Learning rate
  * Batch size : 16이 최적이었음.
  * Seed
  * Hidden Layers
  * Momentum
  * Loss Function
  * K of KNN
* Tuning Techniques
  * Grid Search
  * Random Search
  * Bayesian Optimization
  * Non-Probabilistic
  * Evolutionary Optimization
  * Gradient-based Optimization
  * Early Stopping


### 3. Cross Validation (LSTM에 추천은 안 하는 것 같다 ⬇)
* https://www.quora.com/How-can-I-apply-a-cross-validation-technique-in-an-LSTM-model
* Holdout method
* k-fold cross validation
* Leave-p-out cross validation
* Leave-one-out cross validation
* Stratified k-fold cross validation

### 4. 분류 못한 추가 기법
* RNN 기반 모델의 경우 attention 메커니즘 시도 가능

* Learning Rate Scheduler: 상황에 맞게 가변적인 학습률 적용(StepLR/ MultiStepLR/ ExponentialLR)
* Weight Regularization: optimizer의 weight_decay 파라미터 조정(L1/L2 정형화)
* Weight Initialization: 신경망 가중치 값의 분포가 한쪽으로 쏠리는 현상을 막기 위해 적당하게 가중치를 초기화함(Xavier/Kaming He 초기화)
* Batch Normalization: 신경망 활성화 값 분포를 적당하게 개선(오버피팅 억제 효과를 가지고 있어서 가중치 감소나 드롭아웃 기법의 필요성이 줄어듦
* 결측치 채우기**
* (이번 프로젝트에 적용 가능한지는 모르겠음) Data Augmentation: 알고리즘을 이용해 훈련 데이터 수를 늘리는 방법
* (이번 프로젝트에 적용 가능한지는 모르겠음 )Data Normalization: 입력 데이터에 대해 공간상 분포를 정규화시켜 정확도를 높이는 것
* (피팅은 커녕 오버피팅도 안 일어나서 사용할 일이 있을지 모르겠음) Drop Out: 훈련 데이터에 대한 정확도가 떨어짐
*  출처: http://www.gisdeveloper.co.kr/?p=8443

### 5. 이해 안되는 부분
* 시계열 데이터를 처리하기 위해서는 데이터를 stationary하게 만든 뒤에 AR, MA, ARMA, ARIMA 모델 등을 적용해야 한다.
데이터가 stationary하기 위해서는 시계열 데이터의 평균과 분산이 시간에 따라 일정해야 하고 (래그 h에 따른) 공분산이 일정해야 한다.
이를 검증하기 위해 Augmented Dickey-Fuller Test가 있다(파이썬 statsmodel 패키지에서 adfuller라는 api 제공)

  출처: https://hongl.tistory.com/98
> (검증 방법인데 성능 향상 방법이 맞나...)



* 배치사이즈는 반드시 train,valid,test 셋 모두에 대해서 나누어 떨어져야하므로 1로 두라는 글을 봄.

  출처: https://hhhh88.tistory.com/38
> (1로 두면 성능 안 좋던데...)






In [109]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [110]:
cd /content/drive/MyDrive/Colab Notebooks/DACON_PakChoi

/content/drive/MyDrive/Colab Notebooks/DACON_PakChoi


In [111]:
import random
import pandas as pd
import numpy as np
import os
import glob

import torch
import torch.nn as nn # NN Base code(Class)
import torch.optim as optim # Optimizer
import torch.nn.functional as F # NN Base code(Function)
from torch.utils.data import Dataset, DataLoader # Custom Dataset

from tqdm.auto import tqdm # Progress bar

import warnings
warnings.filterwarnings(action='ignore') 

In [112]:
# TO USE GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Hyperparameter Setting

* epochs: 전체 데이터셋에 대한 학습 횟수
* Learning Rate: 훈련 과정에서의 비용함수 파라미터 조정 비율
* batch size: 한 번에 학습시킬 훈련 샘플 수
* Seed: 난수 생성용

In [113]:
CFG = {
    'EPOCHS':5,
    'LEARNING_RATE':1e-3,
    'BATCH_SIZE':16,
    'SEED':41
}

# Fixed RandomSeed
* Reproducibility 유지를 위해 시드 고정
--> 생성되는 난수 수열을 같게 유지할 수 있음

In [114]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

# Data Pre-processing

In [115]:
all_input_list = sorted(glob.glob('./train_input/*.csv'))
all_target_list = sorted(glob.glob('./train_target/*.csv'))

In [116]:
train_input_list = all_input_list[:50]
train_target_list = all_target_list[:50]

val_input_list = all_input_list[50:]
val_target_list = all_target_list[50:]

# CustomDataset

In [117]:
class CustomDataset(Dataset):
    def __init__(self, input_paths, target_paths, infer_mode):
        self.input_paths = input_paths
        self.target_paths = target_paths
        self.infer_mode = infer_mode
        
        self.data_list = []
        self.label_list = []
        print('Data Pre-processing..')
        for input_path, target_path in tqdm(zip(self.input_paths, self.target_paths)):
            input_df = pd.read_csv(input_path)
            target_df = pd.read_csv(target_path)
            
            input_df = input_df.drop(columns=['시간'])
            input_df = input_df.fillna(0)
            
            input_length = int(len(input_df)/1440)
            target_length = int(len(target_df))
            
            for idx in range(target_length):
                time_series = input_df[1440*idx:1440*(idx+1)].values
                self.data_list.append(torch.Tensor(time_series))
            for label in target_df["rate"]:
                self.label_list.append(label)
        print('Done.')
              
    def __getitem__(self, index):
        data = self.data_list[index]
        label = self.label_list[index]
        if self.infer_mode == False:
            return data, label
        else:
            return data
        
    def __len__(self):
        return len(self.data_list)

In [118]:
train_dataset = CustomDataset(train_input_list, train_target_list, False)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=6)

val_dataset = CustomDataset(val_input_list, val_target_list, False)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=6)

Data Pre-processing..


0it [00:00, ?it/s]

Done.
Data Pre-processing..


0it [00:00, ?it/s]

Done.


# Model Define


In [119]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.lstm = nn.LSTM(input_size=37, hidden_size=256, batch_first=True, bidirectional=False) # RNN or LSTM or GRU
        self.classifier = nn.Sequential(
            nn.Linear(256, 1),
        )
        
    def forward(self, x):
        hidden, _ = self.lstm(x) # RNN or LSTM or GRU
        output = self.classifier(hidden[:,-1,:])
        return output

# Train

In [120]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.L1Loss().to(device)
    
    best_loss = 9999
    best_model = None
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            optimizer.zero_grad()
            
            output = model(X)
            loss = criterion(output, Y)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        val_loss = validation(model, val_loader, criterion, device)
        
        print(f'Train Loss : [{np.mean(train_loss):.5f}] Valid Loss : [{val_loss:.5f}]')
        
        if scheduler is not None:
            scheduler.step()
            
        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
    return best_model

In [121]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.float().to(device)
            Y = Y.float().to(device)
            
            model_pred = model(X)
            loss = criterion(model_pred, Y)
            
            val_loss.append(loss.item())
            
    return np.mean(val_loss)

# Run

In [122]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = None

best_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/101 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

Train Loss : [0.29227] Valid Loss : [0.23710]


  0%|          | 0/101 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

...

  0%|          | 0/13 [00:00<?, ?it/s]

Train Loss : [0.28367] Valid Loss : [0.25243]


  0%|          | 0/101 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

Train Loss : [0.27883] Valid Loss : [0.24393]


# Inference

In [123]:
test_input_list = sorted(glob.glob('./test_input/*.csv'))
test_target_list = sorted(glob.glob('./test_target/*.csv'))

In [124]:
def inference_per_case(model, test_loader, test_path, device):
    model.to(device)
    model.eval()
    pred_list = []
    with torch.no_grad(): # gradient 계산을 수행하지 않게 되어 메모리 사용량을 아껴줌
        for X in iter(test_loader):
            X = X.float().to(device)
            
            model_pred = model(X)
            
            model_pred = model_pred.cpu().numpy().reshape(-1).tolist()
            
            pred_list += model_pred
    
    submit_df = pd.read_csv(test_path)
    submit_df['rate'] = pred_list
    submit_df.to_csv(test_path, index=False)

In [125]:
for test_input_path, test_target_path in zip(test_input_list, test_target_list):
    test_dataset = CustomDataset([test_input_path], [test_target_path], True)
    test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    inference_per_case(best_model, test_loader, test_target_path, device)

Data Pre-processing..


0it [00:00, ?it/s]

Done.
Data Pre-processing..


0it [00:00, ?it/s]

Done.
Data Pre-processing..


...

Done.
Data Pre-processing..


0it [00:00, ?it/s]

Done.
Data Pre-processing..


0it [00:00, ?it/s]

Done.


In [126]:
import zipfile
os.chdir("./test_target/")
submission = zipfile.ZipFile("../submission.zip", 'w')
for path in test_target_list:
    path = path.split('/')[-1]
    submission.write(path)
submission.close()